# Functions and Definitions

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import os
import shutil
import csv
from datetime import datetime
import tensorflow.keras.utils

SEED = 5
np.seed = SEED

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
def create_csv(results, results_dir):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
def shuffle_validation_weighted(base_dir, split = 0.2, reset = False):
    # Create validation_set weighted on number of occurrences in training_set
    train_dir = os.path.join(base_dir, 'training')
    valid_dir = os.path.join(base_dir, 'validation')

    if not reset:
        # First identify training and validation dir
        if not os.path.exists(valid_dir):
            os.makedirs(valid_dir)

        # Count elements in each dir in training
        class_and_card = {name: len(os.listdir(os.path.join(train_dir, name))) for name in os.listdir(train_dir) if
                          os.path.join(train_dir, name)}

        print(class_and_card)
        # Get images per class wrt total images
        class_and_card_validation = {name: int(class_and_card[name] * split) for name in class_and_card}

        print(class_and_card_validation)
        # Select images to move
        for key, item in class_and_card_validation.items():
            source_dir = os.path.join(train_dir, key)
            images = os.listdir(source_dir)
            np.random.shuffle(images)
            target_dir = os.path.join(valid_dir, key)
            if not os.path.exists(target_dir):
                os.makedirs(target_dir)
            for i in range(item):
                shutil.move(os.path.join(source_dir, images[i]), target_dir)
    else:
        # Restore initial state
        # For each class, move images to train_dir in respective folders
        classes = [name for name in os.listdir(valid_dir)]
        for class_name in classes:
            source_dir = os.path.join(valid_dir, class_name)
            images = os.listdir(source_dir)
            target_dir = os.path.join(train_dir, class_name)
            for img in images:
                shutil.move(os.path.join(source_dir, img), target_dir)

In [ ]:
cwd = "./"

# Model and Dataset Definitions

In [ ]:
# We executed a scoped GridSearch on the following parameters
# Dense Layers -> [1, 2, 3] (tested parameters)
# Neurons Number -> [256, 512, 1024]
# Starting Filter Number -> [8, 16, 32, 64]
# Depth -> range(1, 8)
# Image Size -> [256] (same for heigth and width)
# Batch Size -> [8, 16, 32]
# Valid Percentage -> [0.2]
# Starting Learning Rate -> [1e-3, 1e-4]
# LR1 values -> // not tested
# LR2 values -> // not tested
# Below we have the best results
batch_size = 16
img_w, img_h = 256, 256
num_classes = 3
valid_split_perc = 0.2
depth = 4
start_f = 16

In [ ]:
shuffle_validation_weighted(cwd, split=valid_split_perc, reset=False)

{'0': 1900, '1': 1897, '2': 1817}
{'0': 285, '1': 284, '2': 272}


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode="nearest")

train_generator = train_datagen.flow_from_directory(os.path.join(cwd, 'training'),
                                                    target_size=(img_w, img_h),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    seed = SEED)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(os.path.join(cwd, 'validation'),
                                                target_size=(img_w, img_h),
                                                batch_size=batch_size,
                                                shuffle=False,
                                                class_mode='categorical',
                                                seed = SEED)

Found 4773 images belonging to 3 classes.
Found 841 images belonging to 3 classes.


In [ ]:
input_layer = tf.keras.Input(shape=(img_h, img_w, 3))

pre_conv = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3,3),
                                 strides=(1, 1),
                                 padding='valid',
                                 activation='relu')(input_layer)
pre_norm = tf.keras.layers.BatchNormalization()(pre_conv)
pre_pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(pre_norm)

temp_first = pre_pool

# Layers are concatenated.
# 1 Parallel layer > 1x1 | 3x3
# 2 Parallel layer > 1x1 | 5x5
# 1 Parallel layer > 1x1 
# A batch normalisation is added for overfitting
for i in range(depth):
    layer1 = tf.keras.layers.Conv2D(filters=start_f,
                                    kernel_size=(1, 1),
                                    strides=(1, 1),
                                    padding='same',
                                    activation='relu')(temp_first)
    layer2_0 = tf.keras.layers.Conv2D(filters=start_f//4, # Less filters to reduce spatial dimension
                                      kernel_size=(1, 1),
                                      strides=(1, 1),
                                      padding='same',
                                      activation='relu')(temp_first)
    layer2_1 = tf.keras.layers.Conv2D(filters=start_f,
                                      kernel_size=(3, 3),
                                      strides=(1, 1),
                                      padding='same',
                                      activation='relu')(layer2_0)
    layer3_0 = tf.keras.layers.Conv2D(filters=start_f//4,
                                      kernel_size=(1, 1),
                                      strides=(1, 1),
                                      padding='same',
                                      activation='relu')(temp_first)
    layer3_1 = tf.keras.layers.Conv2D(filters=start_f,
                                      kernel_size=(5, 5),
                                      strides=(1, 1),
                                      padding='same',
                                      activation='relu')(layer3_0)

    layer_out = tf.keras.layers.concatenate([layer1, layer2_1, layer3_1], axis = 3)
    batch_norm = tf.keras.layers.BatchNormalization()(layer_out)
    pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(batch_norm)
    temp_first = pooling
    start_f *= 2 # This increases the number of filters

flat_layer = tf.keras.layers.Flatten()(pooling)
dense_0 = tf.keras.layers.Dense(512, activation='relu')(flat_layer)
drop_layer = tf.keras.layers.Dropout(0.3)(dense_0)
dense_1 = tf.keras.layers.Dense(3, activation='softmax')(drop_layer)

model = Model(input_layer, dense_1)

model.summary()

tensorflow.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 254, 254, 16) 448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 254, 254, 16) 64          conv2d_42[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 127, 127, 16) 0           batch_normalization_10[0][0]     
____________________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
cb_early_stopper = EarlyStopping(monitor = 'val_accuracy', patience = 8, mode = 'max')
cb_checkpointer = ModelCheckpoint(filepath = 'best.hdf5', monitor = 'val_accuracy', save_best_only = True, mode = 'max')

# Training

In [ ]:
nb_train_samples = 4773
nb_val_samples = 841

epochs_fine = 200

hist = model.fit(train_generator,
                 steps_per_epoch=nb_train_samples // batch_size,
                 epochs=epochs_fine,
                 validation_data=val_generator,
                 validation_steps=nb_val_samples // batch_size,
                 callbacks=[cb_checkpointer, cb_early_stopper],
                 verbose = 2)

plot_hist(hist)

model.load_weights("best.hdf5")

# Result Evaluation Submission

In [ ]:
model.load_weights('inception_80.hdf5')

test_dir = os.path.join(cwd, 'test')

test_data_gen = ImageDataGenerator(rescale=1./255)

test_gen = test_data_gen.flow_from_directory(test_dir, target_size=(img_h, img_w), 
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 classes = None,
                                                 batch_size=1,
                                                 shuffle=False)
test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}

images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1
  
create_csv(results, cwd)